In [ ]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import time
import traceback

In [ ]:
products = pd.read_pickle("products.pkl")

In [ ]:
products.head()

In [ ]:
import time
import json
import traceback
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def details(products_df):
    results = []

    # Настройка ChromeDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--window-size=1920,1080")

    for idx, row in products_df.iterrows():
        url = row['product_url']
        print(f"\n[INFO] Processing ({idx + 1}/{len(products_df)}): {url}")

        try:
            driver = webdriver.Chrome(options=chrome_options)
            wait = WebDriverWait(driver, 10)
            driver.get(url)
            time.sleep(2)

            try:
                sastav_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Состав")]')))
                driver.execute_script("arguments[0].click();", sastav_tab)
                time.sleep(2)
            except Exception as e:
                print(f"[ERROR] Could not click 'Состав': {e}")

            try:
                show_ingr_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Посмотреть все ингредиенты")]')))
                driver.execute_script("arguments[0].click();", show_ingr_btn)
                time.sleep(2)
            except Exception as e:
                print(f"[WARNING] 'Посмотреть все ингредиенты' not clickable: {e}")

            soup = BeautifulSoup(driver.page_source, "html.parser")

        except Exception as e:
            print(f"[CRITICAL] Could not load page: {e}")
            traceback.print_exc()
            continue
        finally:
            try:
                driver.quit()
            except:
                pass

        # ========== ТВОЙ ПАРСИНГ ==========

        title_tag = soup.select_one("h1.page-title span")
        product_name = title_tag.text.strip() if title_tag else row['name']

        price = soup.select_one(".current-price span")
        price = price.text.strip() if price else "❌"

        description = "❌"
        json_ld = soup.find("script", type="application/ld+json")
        if json_ld:
            try:
                data = json.loads(json_ld.string)
                description = data.get("description", "❌")
            except:
                pass
        if description == "❌":
            desc_block = soup.select_one(".product-description")
            if desc_block:
                description = desc_block.get_text(strip=True)

        ingredients = "❌"
        ingredients_block = soup.select_one("#composition-more")
        if ingredients_block:
            ingredients = ingredients_block.get_text(strip=True)

        characteristics = {
            "Тип кожи": "❌",
            "Тип средства": "❌",
            "По назначению": "❌",
            "Производитель": "❌",
            "Страна": "❌",
            "Объём (мл)": "❌"
        }

        char_section = soup.select("section.product-features dl.data-sheet")
        if char_section:
            dt_dd_pairs = char_section[0].find_all(["dt", "dd"])
            for i in range(0, len(dt_dd_pairs) - 1, 2):
                key = dt_dd_pairs[i].get_text(strip=True)
                value = dt_dd_pairs[i + 1].get_text(separator=", ", strip=True)
                if key in characteristics:
                    characteristics[key] = value

        result = {
            "Product Name": product_name,
            "URL": url,
            "Price": price,
            "Description": description,
            "Ingredients": ingredients
        }
        result.update(characteristics)

        print(f"[OK] Parsed: {product_name} | {price} | {characteristics['Производитель']}")
        results.append(result)

    return pd.DataFrame(results)


In [ ]:
full_df = details(products)

In [ ]:
full_df.to_pickle('data.pkl')